# Extract Building Footprints and Imagery by Region with BRAILS++
This notebook demonstrates how to:
1. Select a location and create a region boundary.
2. Extract building footprints using a selected scraper.
3. Subsample the inventory for faster processing.
4. Retrieve aerial imagery from Google Satellite.
5. Retrieve street-level imagery from Google Streetview.
6. Display inventory information.

## Install BRAILS++

Before running the following cells, install the BRAILS++ package.  

In [ ]:
!pip install brails

In [ ]:
from brails import Importer 
from brails.utils import PlotTools

## Step 0: Modify These Variables Before Running the Notebook
- `LOCATION` : Set the target location for image retrieval (e.g., `'Ft Myers Beach, Florida'`).  
- `FOOTPRINT_SCRAPER` : Specify the scraper class to use (e.g., `'OvertureMapsFootprintScraper'`).  
- `API_KEY` : Provide your Google API key for accessing Street View and other services.

In [ ]:
LOCATION = 'Ft Myers Beach, Florida'
FOOTPRINT_SCRAPER = 'OvertureMapsFootprintScraper'
API_KEY = 'YOUR-API-KEY-HERE'

## Step 1: Create an `Importer` Instance
`Importer` is BRAILS++’s dynamic class loader. In this example, you will use an `Importer` instance to retrieve different components of the BRAILS++ pipeline such as:
- `RegionBoundary` (to define an area of interest)
- Scrapers like `OvertureMapsFootprintScraper` (to get building footprints)
- Imagery providers like `GoogleSatellite` or `GoogleStreetview`

In [ ]:
importer = Importer()

## Step 2: Define the Location and Create a RegionBoundary
The **RegionBoundary** object tells **BRAILS++** the *geographical area* where we want to extract building footprints and/or related data.

**In this example:**
- We obtain the `RegionBoundary` class dynamically using the `Importer` instance (no hardcoded module paths).
- We create a `RegionBoundary` object by passing a dictionary describing the type of location input and the actual data.

**`type` can be one of:**
- `locationName` – a place name, like `'Berkeley, CA'`
- `locationPolygon` – a tuple containing coordinates for a polygon in (lon1, lat1, .... lon_n, lat_n) format

**`data`** holds the actual value for that type.




In [ ]:
region_boundary_class = importer.get_class('RegionBoundary')
region_boundary_object = region_boundary_class(
    {'type': 'locationName', 'data': LOCATION})

## Step 3: Create a Footprint Scraper and Retrieve Building Data

In this step, we set up a **scraper** to retrieve building footprints within the previously defined region.

- First, we dynamically obtain the scraper class from the `Importer` by name (stored in the `scraper` variable). This avoids hardcoding class imports and makes it easy to swap different scraper implementations.
- We create an instance of the scraper, passing configuration parameters. Here, `{"length": "ft"}` specifies that all length-related measurements in the results should be reported in **feet**.
- Using the `get_footprints()` method, we request all building footprints inside the `RegionBoundary` defined earlier. The result is stored in an **inventory** object, which contains:
  - Geometry of each building footprint
  - Metadata such as area, length, and possibly height (depending on the scraper)
- Finally, we print a summary of how many assets (buildings) were found in the given location using the selected scraper.

In [ ]:
scraper_class = importer.get_class(FOOTPRINT_SCRAPER)
scraper = scraper_class({'length': 'ft'})
inventory = scraper.get_footprints(region_boundary_object)
print(f'Number of assets found: {len(inventory.inventory)} for {LOCATION} '
      f'using {FOOTPRINT_SCRAPER}')

## Step 4: Subsample the Inventory for Quick Processing

The **inventory** may contain hundreds or even thousands of buildings, depending on the size and density of the selected region.

To keep this example lightweight and fast:
- We randomly select a fixed number of buildings from the full inventory using `inventory.get_random_sample(nsamples, seed)`.
- Here: `nsamples=20` means exactly **20 buildings** will be chosen. `seed=40` sets the random seed so that the selection is **repeatable** running the notebook again with the same seed will return the same sample.

In [ ]:
small_inventory = inventory.get_random_sample(nsamples=20, seed=40)

## Step 5: Retrieve Aerial Imagery (Google Satellite)
In this cell, we dynamically load and use the `GoogleSatellite` class to fetch satellite images:

1. **Load and instantiate the class**: Retrieve the `GoogleSatellite` class from the `importer` object, allowing dynamic access without a direct import.
2. **Fetch images**: Use the class instance to download satellite images for the items in `small_inventory` and stores them locally in `'tmp/satellite/'`. The resulting images are saved in `images_satellite` for later use.
3. **Show image information**: Displays metadata and basic information about the downloaded satellite images, helping verify the results.

In [ ]:
google_satellite = importer.get_class('GoogleSatellite')()
images_satellite = google_satellite.get_images(
    small_inventory, 'tmp/satellite/')

images_satellite.print_info()
PlotTools.plot_images(images_satellite)

## Step 6: Retrieve Street-Level Imagery (Google Street View)

In this cell, we dynamically load and use the `GoogleStreetview` class to fetch street-level images:
1. **Load and instantiate the class**: Retrieve the `GoogleStreetview` class from the `importer` object, allowing dynamic access without a direct import.
2. **Fetch images**: Use the class instance to download street-level images for the items in `small_inventory` and store them locally in `'tmp/street/'`. The resulting images are saved in `images_street` for later use.
3. **Show image information**: Displays metadata and basic information about the downloaded street-level images, helping verify the results.

In [ ]:
google_street = importer.get_class('GoogleStreetview')({'apiKey': API_KEY})
images_street = google_street.get_images(small_inventory, 'tmp/street/')

images_street.print_info()
PlotTools.plot_images(images_street)